In [18]:
from pathlib import Path
from datetime import datetime, timedelta

def average_latency(sub_log: Path) -> float:
	readings = []
	with sub_log.open() as f:
		for line in f:
			if " - " in line:
				ts, val = line.split(" - ")
				readings.append((datetime.fromisoformat(ts), datetime.fromisoformat(val.strip())))

	# remove last value (is unusally large latency due to timeout)
	readings = readings[:-1]

	# only keep last 50 seconds to be safe
	last_ts = readings[-2][0]
	readings = [(ts, val) for ts, val in readings if ts > last_ts - timedelta(seconds=50)]

	# add 1s to values (cache flushing issue)
	readings = [(ts, val + timedelta(seconds=1))  for ts, val, in readings]


	latency = [(ts-val).total_seconds() for ts, val in readings]

	return sum(latency)/len(latency)

def average_cpu_usage(relay_perf: Path) -> float:
	readings = []
	with relay_perf.open() as f:
		for line in f:
			ts, val = line.split("   ")
			if val == "":
				raise ValueError("empty val")
			readings.append((ts, float(val)))
	percentages = [v for ts, v in readings]

	return sum(percentages)/len(percentages)


def load_measurements(dir: Path):
	measurements = dir / Path("measurements")
	if not measurements.is_dir():
		raise ValueError(f"invalid directory: {measurements}")

	sub_logs = []
	relay_perf = []

	# organize files
	for file in measurements.iterdir():
		if file.name.startswith("clock"):
			sub_logs.append(file)
		if file.name.startswith("relay") and file.name.endswith("cpu_usage.log"):
			relay_perf.append(file)

	sub_avg_latencies = []
	for sub in sub_logs:
		try:
			sub_avg_latencies.append(average_latency(sub))
		except Exception as e:
			pass#print(f"{sub} skipped due to {e}")

	relay_perf_avg = {}
	for relay in relay_perf:
		name = relay.stem.replace("_cpu_usage", "")
		try:
			relay_perf_avg[name] = average_cpu_usage(relay)
		except Exception as e:
			pass#print(f"{name} skipped due to {e}")


	return sub_avg_latencies, relay_perf_avg

load_measurements(Path("/Users/earvarm/Documents/repos/moq-rs/dev/performance-measurement/star"))

([0.012668784313725492,
  0.010849627450980394,
  0.013276803921568627,
  0.01284149019607843,
  0.010979607843137253,
  0.010915686274509806,
  0.009745235294117647,
  0.01319049019607843,
  0.010380039215686275,
  0.010173019607843137,
  0.007728627450980392,
  0.010757568627450982,
  0.00682343137254902,
  0.013177725490196074,
  0.007317529411764705,
  0.0066347450980392166,
  0.010244352941176468,
  0.007359921568627452,
  0.0075295686274509796,
  0.01081992156862745,
  0.013316764705882353,
  0.01097337254901961,
  0.012679039215686276,
  0.006913176470588236,
  0.013270235294117649,
  0.007778156862745098,
  0.010658333333333339,
  0.013121509803921566,
  0.013075,
  0.007646098039215685,
  0.007451470588235293,
  0.00747443137254902,
  0.010667137254901964,
  0.012603411764705882,
  0.01099327450980392,
  0.013188137254901961,
  0.006803196078431372,
  0.007343807692307693,
  0.006770862745098039,
  0.013402862745098044,
  0.013467352941176474,
  0.013025117647058824,
  0.00693

In [20]:
from pprint import pprint
base = Path("/Users/earvarm/Documents/repos/moq-rs/dev/performance-measurement")
for topo in ["full-mesh", "leaf-and-spine", "single-relay", "star", "two-relays"]:
	latencies, relays = load_measurements(base / topo)

	avg_lat = sum(latencies) / len(latencies)
	pprint({
		"name": topo,
		"average_latency": avg_lat,
		"relay_utilizations": relays
	})

{'average_latency': 0.013342964013475941,
 'name': 'full-mesh',
 'relay_utilizations': {'relay1': 0.22258064516129014,
                        'relay2': 0.18860215053763438,
                        'relay3': 0.15526881720430105,
                        'relay5': 0.086236559139785}}
{'average_latency': 0.009718046638150441,
 'name': 'leaf-and-spine',
 'relay_utilizations': {'relay2': 0.16745182012847973,
                        'relay4': 0.07634408602150537}}
{'average_latency': 0.010632838913084468,
 'name': 'single-relay',
 'relay_utilizations': {'relay1': 1.6814432989690717}}
{'average_latency': 0.010619020394984914,
 'name': 'star',
 'relay_utilizations': {'relay1': 0.18283261802575093,
                        'relay2': 0.14120171673819745,
                        'relay4': 0.0641630901287554}}
{'average_latency': 0.011356159513452141,
 'name': 'two-relays',
 'relay_utilizations': {'relay1': 0.04433497536945811,
                        'relay2': 1.0438423645320198}}
